In [2]:
import re
import pandas as pd

from string import punctuation
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

#### Function to preprocess our data, in this step the data is cleaned, removed noise and unnecessary information

In [3]:
class PreProcessor(object):

    #stemmer = nltk.stem.RSLPStemmer()
    tokenizer = TweetTokenizer(reduce_len=True, preserve_case=False)
    special_char = ['$', '%', '&', '*', '(', ')', '_', '-', '+', '=', '{', '[', '}', ']', '~', '.', ',', ';', 'º', 'ª', '°', '¹', '²', '³']

    # UniLex: Método Léxico para Análise de Sentimentos Textuais sobre Conteúdo de Tweets em Português Brasileiro*
     ## (UniLex: Lexical Method for Analysis of Textual Sentiments about Tweet Content in Brazilian Portuguese* )
    stoplist_uniLex = ['a', 'agora', 'ainda', 'alguem', 'algum', 'alguma', 'algumas', 'alguns', 'ampla', 'amplas', 'amplo', 'amplos',
     'ante', 'antes', 'ao', 'aos', 'apos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'ate', 'atraves',
     'cada', 'coisa', 'coisas', 'com', 'como', 'contra', 'contudo', 'da', 'daquele', 'daqueles', 'das', 'de', 'dela',
     'delas', 'dele', 'deles', 'depois', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'deste',
     'destes', 'deve', 'devem', 'devendo', 'dever', 'devera', 'deverao', 'deveria', 'deveriam', 'devia', 'deviam',
     'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'do', 'dos', 'e', 'ela', 'elas', 'ele', 'eles', 'em',
     'enquanto', 'entre', 'era', 'essa', 'essas', 'esse', 'esses', 'esta', 'estamos', 'estao', 'estas', 'estava',
     'estavam', 'estavamos', 'este', 'estes', 'estou', 'eu', 'fazendo', 'fazer', 'feita', 'feitas', 'feito', 'feitos',
     'foi', 'for', 'foram', 'fosse', 'fossem', 'grande', 'grandes', 'ha', 'isso', 'isto', 'ja', 'la', 'lhe', 'lhes',
     'lo', 'mas', 'me', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'minha', 'minhas', 'muita', 'muitas',
     'muito', 'muitos', 'na', 'nao', 'nas', 'nem', 'nenhum', 'nessa', 'nessas', 'nesta', 'nestas', 'ninguem', 'no',
     'nos', 'nossa', 'nossas', 'nosso', 'nossos', 'num', 'numa', 'nunca', 'o', 'os', 'ou', 'outra', 'outras', 'outro',
     'outros', 'para', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante',
     'pode', 'podendo', 'poder', 'poderia', 'poderiam', 'podia', 'podiam', 'pois', 'por', 'porem', 'porque', 'posso',
     'pouca', 'poucas', 'pouco', 'poucos', 'primeiro', 'primeiros', 'propria', 'proprias', 'proprio', 'proprios',
     'quais', 'qual', 'quando', 'quanto', 'quantos', 'que', 'quem', 'sao', 'se', 'seja', 'sejam', 'sem', 'sempre',
     'sendo', 'sera', 'serao', 'seu', 'seus', 'si', 'sido', 'so', 'sob', 'sobre', 'sua', 'suas', 'talvez', 'tambem',
     'tampouco', 'te', 'tem', 'tendo', 'tenha', 'ter', 'teu', 'teus', 'ti', 'tido', 'tinha', 'tinham', 'toda', 'todas',
     'todavia', 'todo', 'todos', 'tu', 'tua', 'tuas', 'tudo', 'ultima', 'ultimas', 'ultimo', 'ultimos', 'um', 'uma',
     'umas', 'uns', 'vendo', 'ver', 'vez', 'vindo', 'vir', 'vos', 'vos']

    # NLTK stopwords + UniLex stopwords
    stoplist = sorted(set(stoplist_uniLex + stopwords.words('portuguese')))
    #stoplist = stopwords.words('portuguese')
    
    def process(self, tweet):
        tweet = self.to_lower(tweet)
        tweet = self.remove_links(tweet)
        tweet = self.remove_mentions(tweet)
        tweet = self.remove_hashtags(tweet)
        tweet = self.remove_numbers(tweet)
        tweet = self.replace_three_or_more(tweet)

        words = self.tokenizer.tokenize(tweet)
        words = self.remove_punctuation(words)
        #words = self.remove_stopwords(words)
        #words = self.applystemmer(words)

        
        processed_words = []
        for word in words:
            if len(word) <= 3:
                word = re.sub('[:;=8][\-=^*\']?[)\]Dpb}]|[cCqd{(\[][\-=^*\']?[:;=8]', 'bom', word)
                word = re.sub('[:;=8][\-=^*\']?[(\[<{cC]|[D>)\]}][\-=^*\']?[:;=8]', 'ruim', word)
                
            if len(word) <= 2:
                word = ''

            for s in self.special_char:
                word = word.replace(s, '')

            processed_words.append(word)

        tweet = ' '.join(processed_words)
        tweet = self.remove_duplicated_spaces(tweet)
        return tweet
    
    def to_lower(self, tweet):
        return tweet.lower()

    def remove_links(self, text):   
        return re.sub(r"http\S+", "", text)
    
    def remove_mentions(self, tweet):
        return re.sub("@\S+", "", tweet)

    def remove_hashtags(self, text):
        entity_prefixes = ['@','#']
        for separator in  punctuation:
            if separator not in entity_prefixes:
                text = text.replace(separator,' ')
        words = []
        for word in text.split():
            word = word.strip()
            if word:
                if word[0] not in entity_prefixes:
                    words.append(word)
        return ' '.join(words)

    def remove_numbers(self, tweet):
        return re.sub("\d+", "", tweet)

    def replace_three_or_more(self, tweet):
        pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
        return pattern.sub(r"\1\1", tweet)

    def remove_duplicated_spaces(self, tweet):
        tweet = tweet.strip()
        return re.sub(" +", " ", tweet)

    def remove_stopwords(self, palavras):
        return [palavra for palavra in palavras if palavra not in self.stoplist]

    def remove_punctuation(self, palavras):
        return [palavra for palavra in palavras if palavra not in list(punctuation)]

In [4]:
preprocessor = PreProcessor()

#### Open the collected data

In [5]:
data = pd.read_csv('tweets.csv', sep=",")

In [6]:
data['preprocessed'] = data['preprocessed'].astype(str) 

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1000 non-null   int64  
 1   text          1000 non-null   object 
 2   preprocessed  1000 non-null   object 
 3   sentiment     0 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 31.4+ KB


#### Passing our data in the process function

In [8]:
for index, row in enumerate(data['text']):
    data.at[index, 'preprocessed'] =  preprocessor.process(str(data.at[index, 'text']))

In [9]:
data['preprocessed'].head(50)

0                não vai com nenhum gente numa pandemia
1     governo libera logo essa acordo furado espera ...
2     viagem nunca dar certo meio pandemia queria festa
3     até porque hospitais estão vazios pandemia con...
4     also foi bem antes pandemia acho tweet referia...
5     reflita pobres miseráveis invisíveis muitos es...
6     sou pessoa mais feliz mundo por ter acompanhad...
7     naoo mais menos mes pandemia surtei voltei com...
8     bolsonarismo disposto derrubar bolsonaro fazer...
9     tudo der certo segunda dose braço pandemia não...
10    discurso anti sus enfraqueceu pandemia sobrara...
11                      rodada pandemia vai ter acabado
12    podemos descarta aderir movimento impeachment ...
13    presa meu antes depois início pandemia pra per...
14    tava muito feia então sarrafo pandemia não tão...
15      previsão pandemia vai acabar rodada brasileirão
16    minha irmã outra mãe thread gente hoje vou exp...
17    para não dizer que não falei das pérolas f

#### Create CSV and saving our preprocessed data

In [10]:
data.to_csv(r'tweets_processed.csv', index = False)